### 📈 Visualizing LFP, MUA, Sniff, and TTL Signals

This script generates time-series plots for electrophysiological recordings, including **Local Field Potentials (LFPs)**, **Multi-Unit Activity (MUA)**, **Sniff Signals**, and **TTL Pulses**. The plots help visualize neural dynamics, respiratory behavior, and event markers across different mice, sessions, and channels.

#### 🗂️ **Data Source:**
The data is loaded from preprocessed `.npy` files:
- **`lfp.npy`**: Local Field Potentials (LFP)
- **`mua.npy`**: Multi-Unit Activity (MUA)
- **`sniff.npy`**: Sniff data (raw)
- **`ttls.npy`**: TTL event markers (raw)

#### 🔍 **Plotting Details:**
- **Number of Segments:** The data is divided into `n_segments` for visualization.  
- **Segment Length:** Each segment is **10 seconds** long.  
- **Sampling Rate:** Signals are sampled at **30 kHz**.  
- **Channels:** LFP and MUA are plotted for each recording channel.

#### 🎨 **Plot Aesthetics:**
- **Consistent Color Palette:**  
  - LFP → Blue (#1f77b4)  
  - MUA → Orange (#ff7f0e)  
  - Sniff → Green (#2ca02c)  
  - TTL → Red (#d62728)  
- **Gridlines:** Added for improved readability.  
- **Shared X-Axis:** All signals are aligned on the time axis (in seconds).  
- **Clean Layout:** Titles, labels, and spacing are optimized for clarity.

#### 💾 **Output Structure:**
The figures are saved in the following format:

```
<fig_dir>
├── <mouse_id>
| ├── <session_id>
| | ├── channel_<channel_num>segment<segment_num>.png
```

Each plot displays:
- **Mouse ID**
- **Session ID**
- **Channel Number**
- **Segment Start Time (in seconds)**

---

This visualization workflow facilitates quick inspection of neural recordings, allowing researchers to identify patterns, artifacts, and key features in the data across multiple sessions and subjects.








In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy import signal
from plotting import plot_raw_signals
import os

In [ ]:
# Specify the directory containing the data
data_dir = r"E:\clickbait-ephys\data\preprocessed"

# List of mice to process
mice = ['6002', '6003', '6000', '6001']

# Specify the directory to save figures
fig_dir = r"E:\clickbait-ephys\figures\data_acquisition\raw_signals_dark"
os.makedirs(fig_dir, exist_ok=True)

In [ ]:

sns.set_context('poster')

plt.style.use('dark_background')

# Constants
n_segments = 3       # Number of segments to plot
length = 3           # Length of each segment (in seconds)
fs = 30000            # Sampling frequency (Hz)
time_axis = np.arange(0, length, 1/fs)  # Time vector for plotting (in seconds)

# Custom color palette
colors = {
    'LFP': '#1f77b4',
    'MUA': '#ff7f0e',
    'Sniff': '#2ca02c',
}


# Looping through mice and sessions
sos = signal.butter(4, [2, 12], 'bandpass', fs=1000, output='sos')
for mouse in mice:
    mouse_sessions = os.listdir(os.path.join(data_dir, mouse))
    for session in tqdm(mouse_sessions, desc=f"Processing mouse {mouse}", unit="session"):

        try:

            # Loading data
            session_path = os.path.join(data_dir, mouse, session)
            lfp = np.load(os.path.join(session_path, 'lfp.npy'), mmap_mode='r', )
            mua = np.load(os.path.join(session_path, 'mua.npy'), mmap_mode='r', )
            sniff = np.load(os.path.join(session_path, 'sniff.npy'), mmap_mode='r', )


            nchannels, num_samples = lfp.shape

            # Create figure directory
            current_fig_dir = os.path.join(fig_dir, mouse, session)
            os.makedirs(current_fig_dir, exist_ok=True)

            # Get equally spaced segment start times
            start_times = np.linspace(0, num_samples - length * fs, n_segments).astype(int)

            # Plotting signals for each channel and segment
            theta = signal.sosfiltfilt(sos, lfp).astype(np.int16)
            for channel in range(nchannels):
                channel_dir = os.path.join(current_fig_dir, f"channel_{channel}")
                os.makedirs(channel_dir, exist_ok=True)
                for seg in start_times:
                    save_path = os.path.join(channel_dir, f"segment_{seg}.png")
                    plot_raw_signals(lfp, theta, mua, sniff, channel, seg, mouse, session, save_path, time_axis, length, fs, colors)

        except Exception as e:
            print(f"Error processing {mouse} | Session {session}: {e}")
            continue